# Softmax
Berikut adalah persamaan dari softmax function
$S(y_i) = \frac{e^{y_i}}{\sum_{j=1}^{n}e^{y_j}}$

**Berikut beberapa hal penting terkait softmax:**
- Softmax function atau ***normalized exponential function*** adalah generalisasi dari fungsi logistik terhadap beberapa dimensi
- Kerap digunakan sebagai fungsi aktivasi untuk menghasilkan probabilitas (antara 0 dan 1)
- Input dari softmax adalah sebuah vektor yang berisi nilai dari suatu variabel, menormalisasikannya menjadi distribusi probabilitas



<div>
<img src="../assets/softmaxlayer.png" width="500"/>
</div>

**Perhatikan ilustrasi diatas ini:**
- Setiap nilai akan di ***squashed*** menjadi nilai probabilitas
- Misalnya nilai terendah (0.1) menjadi 0.1 dan nilai tertinggu (2) menjadi 0.7

In [1]:
import torch
import torch.nn as nn
import numpy as np

### Fungsi Softmax Tanpa Torch

In [2]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)


x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print(f'Input: {x}')
print(f'Output: {outputs}')

Input: [2.  1.  0.1]
Output: [0.65900114 0.24243297 0.09856589]


### Fungsi Softmax Dengan Torch
```torch.softmax(input, dimensi)```

In [3]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print(f'Input: {x}')
print(f'Output: {outputs}')

Input: tensor([2.0000, 1.0000, 0.1000])
Output: tensor([0.6590, 0.2424, 0.0986])


## Ilustrasi Softmax
Sehingga dapat diamati, jika softmax digunakan dalam NN layers, maka implementasinya akan seperti gambar di bawah ini.

<div>
<img src="../assets/softmax2.jpeg" width="700"/>
</div>

Sumber: [Towards Data Science](https://towardsdatascience.com/cross-entropy-loss-function-f38c4ec8643e#:~:text=than%20in%202.-,Cross%2DEntropy%20Loss%20Function,from%20the%20actual%20expected%20value.)

# Cross-Entropy Loss
Kita sebelumnya memahami bahwa softmax mengubah nilai dari suatu variabel menjadi distribusi probabilitas. Tujuan dari cross-entropy loss adalah untuk menghitung error dari softmax. Cross entropy menggunakan hasil dari softmax sebagai input dan mengukur jarak dari nilai kebenaran (truth values).

**Cross-Entropy Loss Equation**
$D(\widehat{y}, y) = -\sum_{i=1}^{n}y_i\log(\hat{y}_i)$

**Contoh 1:**
$Y = [1,0,0]$
$\hat{Y} = [0.7,0.2,0.1]$
$D(\hat{Y}, Y) = 0.35$
**Contoh 2:**
$Y = [1,0,0]$
$\hat{Y} = [0.1,0.3,0.6]$
$D(\hat{Y}, Y) = 2.30$



### Implementasi tanpa Torch

In [4]:
def cross_entropy_loss(sebenarnya, prediksi):
    return -np.sum(sebenarnya * np.log(prediksi))


Y = np.array([1.0, 0.0, 0.0])
Y_prediksi = np.array([0.7, 0.2, 0.1])
Y_prediksi_2 = np.array([0.1, 0.3, 0.6])
Y_prediksi_perfect = np.array([0.999998, 0.000001, 0.000001]) # contoh distribusi yang sangat mirip dengan `Y`

loss_1 = cross_entropy_loss(Y, Y_prediksi)
loss_2 = cross_entropy_loss(Y, Y_prediksi_2)
loss_3 = cross_entropy_loss(Y, Y_prediksi_perfect)

print(f'Loss 1: {loss_1:.3f}')
print(f'Loss 2: {loss_2:.3f}')
print(f'Loss 3: {loss_3:.3f}')  # karena distribusi sangat mirip, seharusnya nilai loss sangat kecil mendekati 0

Loss 1: 0.357
Loss 2: 2.303
Loss 3: 0.000


### Implementasi dengan Torch

```nn.CrossEntropyLoss()```
- CrossEntropyLoss adalah loss function yang menggunakan softmax sebagai input dan menghitung error dari softmax
- Pada fungsi ini, softmax akan dihitung secara otomatis
- Softmax tidak perlu ditempatkan lagi di layer sebelumnya
- ```Y``` harus berupa class labels, dapat juga dalam bentuk [one-hot-encoding](https://en.wikipedia.org/wiki/One-hot) atau juga probabilitas kelas
- ```Y_prediksi``` harus berupa nilai yang belum di-softmax. [Lihat dokumentasi](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)

In [5]:
loss = nn.CrossEntropyLoss()
Y = torch.tensor([0])
Y_prediksi = torch.tensor([[2.0, 1.0, 0.1]])  # Raw Value tanpa Softmax
Y_prediksi_2 = torch.tensor([[0.5, 2.0, 0.3]])  # Raw Value tanpa Softmax
Y_prediksi_perfect = torch.tensor([[10.0, 0.3, 0.1]]) # contoh distribusi yang sangat mirip dengan `Y` (raw value)

loss_1 = loss(Y_prediksi, Y)
loss_2 = loss(Y_prediksi_2, Y)
loss_3 = loss(Y_prediksi_perfect, Y)

print(f'Loss 1: {loss_1.item():.3f}')
print(f'Loss 2: {loss_2.item():.3f}')
print(f'Loss 3: {loss_3.item():.3f}')  # karena distribusi sangat mirip, seharusnya nilai loss sangat kecil mendekati 0

Loss 1: 0.417
Loss 2: 1.841
Loss 3: 0.000


In [6]:
# contoh dengan target merupakan one-hot-encoding
# --- seharusnya hasil sama dengan kode diatas ---
Y = torch.tensor([[1.0, 0.0, 0.0]], dtype=torch.float)
Y_prediksi = torch.tensor([[2.0, 1.0, 0.1]])  # Raw Value tanpa Softmax
Y_prediksi_2 = torch.tensor([[0.5, 2.0, 0.3]])  # Raw Value tanpa Softmax
Y_prediksi_perfect = torch.tensor([[10.0, 0.3, 0.1]]) # contoh distribusi yang sangat mirip dengan `Y` (raw value)

loss_1 = loss(Y_prediksi, Y)
loss_2 = loss(Y_prediksi_2, Y)
loss_3 = loss(Y_prediksi_perfect, Y)

print(f'Loss 1: {loss_1.item():.3f}')
print(f'Loss 2: {loss_2.item():.3f}')
print(f'Loss 3: {loss_3.item():.3f}')  # karena distribusi sangat mirip, seharusnya nilai loss sangat kecil mendekati 0

Loss 1: 0.417
Loss 2: 1.841
Loss 3: 0.000


### Mengambil prediksi terbaik
Menggunakan ```torch.max``` untuk mendapatkan prediksi terbaik. Fungsi ini mengembalikan nilai maksimum dan index maksimum.

Format input : ```torch.max(input, dim)```

In [7]:
_, prediksi1 = torch.max(Y_prediksi, 1)
_, prediksi2 = torch.max(Y_prediksi_2, 1)
_, prediksi3 = torch.max(Y_prediksi_perfect, 1)
print(f'Prediksi-1 terbaik adalah elemen ke: {prediksi1.item()}')
print(f'Prediksi-2 terbaik adalah elemen ke: {prediksi2.item()}')
print(f'Prediksi-3 terbaik adalah elemen ke: {prediksi3.item()}')

Prediksi-1 terbaik adalah elemen ke: 0
Prediksi-2 terbaik adalah elemen ke: 1
Prediksi-3 terbaik adalah elemen ke: 0
